# 주성분 줄이기 차이 비교
### 선형적으로 구분되지 않는 데이터의 차원 축소

In [1]:
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_circles

In [2]:
# 선형적으로 구분되지 않는 데이터 만들기
features, _ = make_circles(n_samples=1000, random_state=777, noise=0.1, factor=0.1)
print(features)

[[-0.02313275  0.15282194]
 [ 0.8626983   0.46556009]
 [ 0.05905661 -0.05048386]
 ...
 [-0.10071513 -0.1814104 ]
 [ 0.10461224  0.08710802]
 [ 0.28894733  0.97460534]]


* 커널 함수는 선형적으로 구분되지 않는 데이터를 선형적으로 구분되는 고차원으로 투영시켜준다.
* 사이킷런의 kernelPCA의 kernel 매개변수의 값 -rbf(가우시안 방사 기저 함수 커널), ploy(다항식 커널), sigmoid, 선형 투영(linear)로 지정
* 여러 가지 커널과 매개변수 조합으로 머신러닝 모델을 여러 번 훈련시켜 가장 높은 예측 성능을 만드는 값의 조합을 찾아야 한다.
* 커널 트릭은 실제 고차원으로 데이터를 변환하지 않으면서 고차원 데이터를 다루는 듯한 효과를 낸다.

데이터를 가우시안 RBF 커널을 사용하여 변환한 결과이며, 차원 수가 1차원으로 감소된 데이터.

#### 방사 기저 함수
데이터 간의 유사도를 측정하기 위해 주로 사용되는 함수 중 하나. RBF는 두 데이터 포인트 사이의 거리가 멀어질수록 값이 작아지는 형태로, 가우시안 함수를 대표적으로 사용한다.

In [ ]:
# 방사 기저 함수 (radius basis function RBF)를 사용하여 커널 PCA를 적용
kpca = KernelPCA(kernel='rbf', gamma=15, n_components=1)
# Kernel rbf : 가우시안 rbf 커널
features_kpca = kpca.fit_transform(features)
print("원본 특성 개수 : ", features.shape[1])
print("줄어든 특성 개수 : ", features_kpca.shape[1])

In [11]:
# 선형 판별 분석 LDA
from sklearn import datasets
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [12]:
iris = datasets.load_iris()    # 붖꽃 데이터셋 로드
features = iris.data
target = iris.target

In [13]:
# LDA 객체 만들고 실행하여 특성을 반환한다.
lda = LinearDiscriminantAnalysis(n_components=1)
features_lda = lda.fit(features, target).transform(features)

print("원본 특성 개수", features.shape[1])
print("줄어든 특성 개수", features_lda.shape[1])

원본 특성 개수 4
줄어든 특성 개수 1


In [14]:
# 행렬 분해를 사용하여 특성 축소
from sklearn.decomposition import NMF
from sklearn import datasets
import numpy as np

In [19]:
digits = datasets.load_digits()      # 데이터 로드
features = digits.data               # 특성 행렬을 로드

nmf = NMF(n_components = 10, random_state = 1)
# NMF 생성 / 위의 코드에서는 10개의 차원으로 변환하도록 설정

features_nmf = nmf.fit_transform(features)       # 학습

print("원본 특성 개수 : ", features.shape[1])
print("줄어든 특성 개수 : ", features_nmf.shape[1])

원본 특성 개수 :  64
줄어든 특성 개수 :  10


C:\Users\smjin\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF : 원본 데이터셋의 특성으 양수로만 분해하는 기법. 입력 데이터셋의 모든 값이 양수일 경우 적용 가능

In [20]:
nmf.components_.shape   # H 행렬
np.all(nmf.components_ >= 0)    # 모두 양수인지 확인

# 원본 데이터를 복원하려면 변환된 행렬 W와 성분 행렬 H를 점곱한다.
# NMF 모델에 의해 변환된 특성 features_nmf를 사용항 원본 데이터 features의 근사값을 계산하고, 이를 이용해 근사 오차를 계산
np.mean(features - np.dot(features_nmf, nmf.components_))

-0.19806745373965523

In [21]:
# NMF 클래스의 solver 매개변수의 기본값은 mu로 좌표하강법을 사용
nmf_mu = NMF(n_components=10, solver='mu', random_state=1)
features_nmf_mu = nmf_mu.fit_transform(features)
print(np.mean(features - np.dot(features_nmf_mu, nmf_mu.components_)))

-0.22082025790022983


features와 NMF 알고리즘으로 근사한 결과 행렬간의 MSE를 계산. 이 값을 통해 근사가 얼마나 잘 이루어졌는지를 평가할 수 있음